Este notebook destina-se a refinar as funções geradas no notebook raw para que estas sejam acessáveis de maneira simples e possam ser utilizadas com outros dataframes.

# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt

# Funções

In [2]:
def get_data(filmes_path, notas_path):
        # lendo os dados
        filmes = pd.read_csv(filmes_path)
        notas = pd.read_csv(notas_path)

        # alterando os nomes das colunas
        filmes.columns = ['filmeId', 'titulo', 'generos']
        notas.columns = ['usuarioId', 'filmeId', 'nota', 'momento']
        notas.drop(columns = ['momento'], inplace = True)
        return filmes, notas

In [3]:
def aux_df(df_notas, df_filmes):
        # criando tabela única para melhor análise
        df = pd.merge(df_notas, df_filmes, how = 'left', right_on = 'filmeId', left_on = 'filmeId')
        # criando coluna com a quantidade de notas atribuídas aos filmes
        df_aux = df.groupby(by = 'filmeId').agg({'nota':'count'}).reset_index()
        # juntando o dataframe inicial com a coluna de quantidade de notas
        df = pd.merge(df, df_aux, how = 'left', right_on = 'filmeId', left_on = 'filmeId')
        # renomeando colunas
        df.rename(columns = {'nota_x':'nota', 'nota_y':'total_de_votos'}, inplace = True)
        # obtendo os filmes mais votados pelas pessoas e suas notas médias
        df_aux2 = df.groupby(by = ['titulo', 'filmeId']).agg({'total_de_votos':'count', 
        'nota':'mean'}).reset_index().sort_values(by = 'total_de_votos', ascending = False)
        df_aux2.rename(columns = {'nota':'nota_media'}, inplace = True)
        # filtrando colunas
        df_aux3 = df_aux2[['filmeId', 'nota_media']]
        # dataframe na forma final
        df = pd.merge(df, df_aux3, how = 'left', right_on = 'filmeId', left_on = 'filmeId')

        return df

In [4]:
# função que calcula a distância euclidiana entre dois vetores
def distancia_de_vetores(vetor_a, vetor_b):
    dist = np.linalg.norm(vetor_a - vetor_b)
    return dist

In [5]:
# função que retorna dataframe com os coluna de notas e index como o id do filme
def notas_do_usuario(data_frame, usuario_id):
    # fltra o dataframe pelo usuarioId
    notas_do_usuario = data_frame[data_frame['usuarioId'] == usuario_id]
        
    # setta o index como a coluna de filmeId
    notas_do_usuario = notas_do_usuario[['filmeId', 'nota']].set_index(['filmeId'])
    return notas_do_usuario

In [6]:
def distancia_de_usuarios(usuario_id1, usuario_id2):
        # trás notas dos usuários 1 e 2
        notas1 = notas_do_usuario(data_frame = df, usuario_id = usuario_id1)
        notas2 = notas_do_usuario(data_frame = df, usuario_id = usuario_id2)

        # trazer somente os filmes que os dois usuários assistiram
        diferencas = notas1.join(notas2, lsuffix = '_esquerda', rsuffix = '_direita').dropna()

        # calcula a distância entre os dois usuários
        distancia = distancia_de_vetores(diferencas['nota_esquerda'], diferencas['nota_direita'])
        return [usuario_id1, usuario_id2, distancia]

In [7]:
def distancia_de_todos(self, id_usuario):
        self.notas
        self.distancia_de_usuarios()
        todos_os_usuarios = notas['usuarioId'].unique()
        distancias = [distancia_de_usuarios(id_usuario, usuario_id) for usuario_id in todos_os_usuarios]
        distancias_de_todos = pd.DataFrame(distancias, columns = ['voce', 'outra_pessoa', 'distancia'])
        return distancias_de_todos

In [8]:
def distancia_de_usuarios(usuario_id1, usuario_id2, minimo = 5):
        # trás notas dos usuários 1 e 2
        notas1 = notas_do_usuario(data_frame = df, usuario_id = usuario_id1)
        notas2 = notas_do_usuario(data_frame = df, usuario_id = usuario_id2)

        # trazer somente os filmes que os dois usuários assistiram
        diferencas = notas1.join(notas2, lsuffix = '_esquerda', rsuffix = '_direita').dropna()

        if(len(diferencas) < minimo):
            return None

        # calcula a distância entre os dois usuários
        distancia = distancia_de_vetores(diferencas['nota_esquerda'], diferencas['nota_direita'])
        return [usuario_id1, usuario_id2, distancia]

In [9]:
def mais_proximos_de(self, id_usuario, n_mais_proximos = 5, numero_de_usuarios_a_analisar = None):
        self.distancia_de_todos()
        distancias = distancia_de_todos(id_usuario, numero_de_usuarios_a_analisar = numero_de_usuarios_a_analisar)
        nearest = distancias.sort_values(by = 'distancia')
        nearest = distancias.set_index('outra_pessoa').drop(id_usuario, errors = 'ignore')
        
        return nearest

In [10]:
def distancia_de_todos(id_usuario, numero_de_usuarios_a_analisar = None):
        todos_os_usuarios = notas['usuarioId'].unique()
        if numero_de_usuarios_a_analisar:
            todos_os_usuarios = todos_os_usuarios[:numero_de_usuarios_a_analisar]
        distancia_a_todos = [distancia_de_usuarios(id_usuario, usuario_id) for usuario_id in todos_os_usuarios]
        distancia_a_todos = list(filter(None, distancia_a_todos))
        distancia_a_todos = pd.DataFrame(distancia_a_todos, columns = ['usuario', 'outro_usuario', 'distancia'])
        return distancia_a_todos

In [11]:
def sugere_para(data_frame, id_usuario, n_mais_proximos = 10, numero_de_usuarios_a_analisar = None):
        # suas notas
        notas_de_usuario = notas_do_usuario(data_frame, id_usuario)

        # indice do filme visto por ti
        filmes_vistos = notas_de_usuario.index

        # pessoas similares
        similares = mais_proximos_de(id_usuario, n_mais_proximos, numero_de_usuarios_a_analisar)
        similares = similares.sort_values(by = 'distancia')
        similares = similares.head(n_mais_proximos)
        usuarios_similares = similares.index
        notas_dos_similares = notas.set_index(['usuarioId']).loc[usuarios_similares]
        recomendacoes = notas_dos_similares.groupby(by = 'filmeId').mean()[['nota']]
        recomendacoes = recomendacoes.sort_values(by = 'nota', ascending = False)
        recomendacoes = recomendacoes.join(filmes)
        return recomendacoes

# Chamando as funções

In [12]:
filmes, notas = get_data(filmes_path = 'movies.csv', notas_path = 'ratings.csv')

In [13]:
filmes.head()

,filmeId,titulo,generos
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
notas.head()

,usuarioId,filmeId,nota
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [15]:
df = aux_df(df_notas = notas, df_filmes = filmes)

In [16]:
df.head()

,usuarioId,filmeId,nota,titulo,generos,total_de_votos,nota_media
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215,3.920930
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,52,3.259615
2,1,6,4.0,Heat (1995),Action|Crime|Thriller,102,3.946078
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,203,3.975369
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,204,4.237745


In [17]:
notas_do_usuario_1 = notas_do_usuario(data_frame = df, usuario_id = 1)

In [18]:
notas_do_usuario_2 = notas_do_usuario(data_frame = df, usuario_id = 2)

In [19]:
notas_do_usuario_1.head()

,nota
filmeId,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0


In [20]:
notas_do_usuario_1.head()

,nota
filmeId,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0


In [21]:
distancia_de_usuarios = distancia_de_usuarios(usuario_id1 = 1, usuario_id2 = 2, minimo = 5)

In [22]:
distancia_de_usuarios

In [23]:
def distancia_de_todos(id_usuario, numero_de_usuarios_a_analisar = None):
        todos_os_usuarios = notas['usuarioId'].unique()
        if numero_de_usuarios_a_analisar:
            todos_os_usuarios = todos_os_usuarios[:numero_de_usuarios_a_analisar]
        distancia_a_todos = [distancia_de_usuarios(id_usuario, usuario_id) for usuario_id in todos_os_usuarios]
        distancia_a_todos = list(filter(None, distancia_a_todos))
        distancia_a_todos = pd.DataFrame(distancia_a_todos, columns = ['usuario', 'outro_usuario', 'distancia'])
        return distancia_a_todos

In [24]:
distancia_a_todos

NameError: name 'distancia_a_todos' is not defined

In [ ]:
sugere_para(data_frame = df, id_usuario = 1, n_mais_proximos = 10, numero_de_usuarios_a_analisar = 50)

AttributeError: 'int' object has no attribute 'distancia_de_todos'